# Vérifier et préparer votre environnement de projet

## Vérifier votre environnement

In [1]:
!gcloud config get-value core/account

pascalr@google.com


In [2]:
!gcloud config get-value core/project

pascal-demo


## Vérifier que les APIs sont activées

In [3]:
!gcloud services list --enabled --filter speech

NAME                         TITLE
speech.googleapis.com        Cloud Speech-to-Text API
texttospeech.googleapis.com  Cloud Text-to-Speech API


## Vérifier que les API Speech-to-Text et Text-to-Speech sont bien activées

In [4]:
!gcloud services list --enabled --filter=speech

NAME                         TITLE
speech.googleapis.com        Cloud Speech-to-Text API
texttospeech.googleapis.com  Cloud Text-to-Speech API


## Vérifier que l'API Translation est bien activée

In [5]:
!gcloud services list --enabled --filter=translate

NAME                      TITLE
automl.googleapis.com     Cloud AutoML API
translate.googleapis.com  Cloud Translation API


## Activer les APIs éventuellement non activées
Si certaines APIs ne sont pas encore activées, retournez sur l'interface web de la console GCP, cliquez sur __Navigation Menu__ (icône dite "Burger menu") > __APIs & Services__ > __Library__.

Cherchez la ou les API(s) non encore activées et cliquez sur __Enable__.

## Déclarer votre clé d'API
Retrouvez la valuer de votre clé d'API :
- depuis l'interface graphique de la console GCP :
cliquez sur __Navigation Menu__ (icône dite "Burger menu") > __APIs & Services__ > __Credentials__

- depuis le fichier dans lequel vous l'avez enregistrée :
Dans __Cloud Shell__ tapez la commende :

```cat My-API-key.txt```

Puis copiez-collez la valeur ci-dessous :

In [ ]:
%env API_KEY <valeur>

In [3]:
PROJECT = !gcloud config get-value core/project
%env GOOGLE_CLOUD_PROJECT {PROJECT[0]}

env: GOOGLE_CLOUD_PROJECT=pascal-demo


In [ ]:
!echo $GOOGLE_CLOUD_PROJECT

# Transcrire un fichier audio vocal vers du texte avec Speech-to-Text API

## Créer un service account et télécharger la clé

In [5]:
!gcloud iam service-accounts create my-lab-sa --display-name "my lab service account"

Created service account [my-lab-sa].


In [7]:
!gcloud iam service-accounts keys create ~/key.json --iam-account my-lab-sa@${GOOGLE_CLOUD_PROJECT}.iam.gserviceaccount.com

created key [1fb39498cbfd0cca2662a6460a31a1509a776f78] of type [json] as [/home/jupyter/key.json] for [my-lab-sa@pascal-demo.iam.gserviceaccount.com]


In [8]:
%env GOOGLE_APPLICATION_CREDENTIALS="/home/jupyter/key.json"

env: GOOGLE_APPLICATION_CREDENTIALS="/home/jupyter/key.json"


## Utiliser la librairie client Python 

Vous allez lancer la transcription d'un [autre fichier sonore vocal](https://storage.googleapis.com/speech-language-samples/fr-sample.flac) (cliquez sur le lien pour l'écouter).

### Vérifier d'abord que le module google-cloud-speech est bien installé :

In [10]:
!pip freeze | grep speech
!echo $GOOGLE_APPLICATION_CREDENTIALS

google-cloud-speech==1.3.2
"/home/jupyter/key.json"


### Ensuite, préparer l'appel à l'API via le client Python :

In [34]:
from google.cloud import speech

#creds = service_account.Credentials.from_service_account_file("/home/jupyter/key.json") 
client = speech.SpeechClient.from_service_account_json('/home/jupyter/key.json')

gcs_uri = "gs://speech-language-samples/fr-sample.flac"

audio = speech.types.RecognitionAudio(uri=gcs_uri)
config = speech.types.RecognitionConfig(
    encoding=speech.types.RecognitionConfig.AudioEncoding.FLAC,
    sample_rate_hertz=44100,
    language_code="fr",
)

### Lancer l'opération en mode syncrhone et afficher la meilleure réponse :

In [35]:
response = client.recognize(config=config, audio=audio)
for result in response.results:
    # The first alternative is the most likely one for this portion.
    print(u"Transcript: {}".format(result.alternatives[0].transcript))
    print("Confidence: {}".format(result.alternatives[0].confidence))

Transcript: maître corbeau sur un arbre perché Tenait dans son bec un fromage
Confidence: 0.9287356734275818


### Lancer l'opération en mode asyncrhone et afficher la meilleure réponse :

In [29]:
operation = client.long_running_recognize(
    config=config, audio=audio
)
print("Waiting for operation to complete...")
response = operation.result(timeout=90)

Waiting for operation to complete...


In [30]:
for result in response.results:
    # The first alternative is the most likely one for this portion.
    print(u"Transcript: {}".format(result.alternatives[0].transcript))
    print("Confidence: {}".format(result.alternatives[0].confidence))

Transcript: maître corbeau sur un arbre perché tenait en son bec un fromage
Confidence: 0.9385557770729065
